In [5]:
# Pyspark should be installed as a dependency
import mlflowrate

In [6]:
mlflowrate.

SyntaxError: invalid syntax (<ipython-input-6-e2916bc76823>, line 1)

In [ ]:
# Load datas into notebook
df_01 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/newdump_01.csv')
df_02 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/newdump_02.csv')
df_OW1ql = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/Qliq.csv')
df_records = spark.read.format('csv').options(header='true', inferSchema='true', delimiter='|', encoding='iso-8859-1').load('/FileStore/tables/interences_filtered.csv')
test_OW1 = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/OW1test_1.csv')

# Convert date columns to datetime formats
# Rename columns so that it fits with flow package.
df_01 = df_01.select(
      F.to_timestamp(F.col("ts").cast("string"), "dd-MMM-yy HH:mm:ss").alias("datetime"),
      df_01["name"].alias("tag"),
      df_01["value"])

df_02 = df_02.select(
      F.to_timestamp(F.col("ts").cast("string"), "dd-MMM-yy HH:mm:ss").alias("datetime"),
      df_02["name"].alias("tag"),
      df_02["value"])

df_OW1ql = df_OW1ql.select(
      F.to_timestamp(F.col("DATE").cast("string"), "MM/dd/yyyy").alias("datetime"),
      *[feat for feat in df_OW1ql.columns if feat not in ["DATE"]])

test_OW1 = test_OW1.select(
      F.to_timestamp(F.col("DATE").cast("string"), "MM/dd/yyyy HH:mm").alias("datetime"),
      *[n for n in test_OW1.schema.names if not n == "DATE"])

df_records = df_records.select(
      F.to_timestamp(F.col("Date").cast("string"), "MM/dd/yyyy").alias("datetime"),
      *[n for n in df_records.schema.names if not n == "Date"])

In [ ]:

#Create dictionaries for translating data tagnames
do = {"OW1" : ["EXAMPLE", "EXAMPLE"],
      "OW2" : ["EXAMPLE", "EXAMPLE"],
      "OW3" : ["EXAMPLE"]}

nn = {"EXAMPLE" : "WHP",
      "EXAMPLE" : "WHT",
      "EXAMPLE" : "GLR",
      "EXAMPLE" : "GLP",
      "EXAMPLE" : "DHP",
      "EXAMPLE" : "DHT",
      "EXAMPLE" : "Choke",
      "EXAMPLE" : "ASD"}

# Collect up data into a dictionary with corresponding names
data = {"df_01":df_01, "df_02":df_02, "df_OW1ql":df_OW1ql, "test_OW1":test_OW1, "df_records":df_records}

# Make new workflow object by passing in the data dictionary 
flow = WorkFlow(data)

# Cache data to speed up spark queries
flow.data.cache_data("df_01", "df_02", "df_OW1ql", "test_OW1", "df_records")

# Merge dataframes vertically
flow.data.merge_data("df_dump", "df_01", "df_02", axis=0)

# select the stored data, and remove nulls
flow.data.clean_data("df_dump", remove_nulls=True)

# select the stored data, and reorganise it to more dictionaries
flow.data.organise_data("df_dump", "date_tag_val_col", distinct_oilwells=do, change_sensor_names=nn) 

# We get three new datasets OW1. OW2. OW3. but their dataframes will be out of shape
flow.data.clean_data("OW1", avg_over="day", is_dict=True)  # average the data on the dict format
flow.data.organise_data("OW1", "dict_col")  # reorganise the data again so the dictionary matches the dataframe
flow.data.set_organised("OW1") # Tell the class that this data is ready for dataset assembly
flow.data.cache_data("OW1") # Cache data frame for fast querying
flow.data.status("OW1") # check the status of the data

flow.data.clean_data("df_OW1ql", remove_nulls=True) # remove nulls in the data
flow.data.edit_col("df_OW1ql", "Daily liquid rate [Sm3/d]", newname="Qliq") # edit the column files to have a new name
flow.data.select_col("df_OW1ql", "Qliq") # reselect the data frame to only be the given sample
flow.data.organise_data("df_OW1ql", "mult_col") # reorganise data so that it we have a correponding group dict format
flow.data.set_organised("df_OW1ql") # Tell the class that this data is ready for assembly
flow.data.cache_data("df_OW1ql") # cache data frame for fast querying
flow.data.status("df_OW1ql") # check the status of the data

flow.data.drop_col("test_OW1", "WELLNAME", "NUMBER", "Qo", "Qg", "Qw", "GOR", "WOR", "WTC", "PI", "Pres", "DCP", "ChokeD", "DCTcalc")
flow.data.clean_data("test_OW1", remove_nulls=True)
flow.data.clean_data("test_OW1", char_col="WHT", remove_char="-")
flow.data.edit_col("test_OW1", "pbh", newname="DHP")
flow.data.edit_col("test_OW1", "Qgl", newname="GLR")
flow.data.edit_col("test_OW1", "Q liq", newname="Qliq")
flow.data.edit_col("test_OW1", "WHT", typ="double")
flow.data.organise_data("test_OW1", "mult_col")
flow.data.set_organised("test_OW1")
flow.data.cache_data("test_OW1")


In [ ]:
flow.next_phase()
# make sets first, these are basically dataframes.
flow.datasets.make_set("Sep", align_dates="day", feats={"OW1": ["GLP", "ASD"], "test_OW1": ["WHP", "Choke", "GLR", "DHP", "Qliq", "WHT"]})
flow.datasets.make_set("Field", align_dates="day", feats={"OW1": ["DHP", "WHT", "GLR", "ASD", "WHP", "GLP", "Choke"], "df_OW1ql":["Qliq"]})

flow.datasets.make_set("Sep_2018", align_dates="day", feats={"OW1": ["GLP", "ASD"], "test_OW1": ["WHP", "Choke", "GLR", "DHP", "Qliq", "WHT"]})
flow.datasets.make_set("Field_2018", align_dates="day", feats={"OW1": ["DHP", "WHT", "GLR", "ASD", "WHP", "GLP", "Choke"], "df_OW1ql":["Qliq"]})

flow.datasets.make_set("OrigSep", align_dates="day", feats={"test_OW1": ["WHP", "Choke", "GLR", "DHP", "Qliq", "WHT"]})
flow.datasets.make_set("OrigField", align_dates="day", feats={"OW1": ["DHP", "WHT", "GLR", "WHP", "Choke"], "df_OW1ql":["Qliq"]})

flow.datasets.make_set("OrigSep_2018", align_dates="day", feats={"test_OW1": ["WHP", "Choke", "GLR", "DHP", "Qliq", "WHT"]})
flow.datasets.make_set("OrigField_2018", align_dates="day", feats={"OW1": ["DHP", "WHT", "GLR", "WHP", "Choke"], "df_OW1ql":["Qliq"]})

In [ ]:
flow.datasets.cache_data("Sep", "Field")
condition = lambda df : df.where(df.Qliq <= 50)
flow.datasets.append_rows("Field", condition, "Sep")
flow.datasets.date_range("OrigSep_2018", "2018-06-01", "2020-01-01")
flow.datasets.date_range("OrigField_2018", "2018-06-01", "2020-01-01")

condition = lambda df : df.where(df.Qliq <= 50)
flow.datasets.append_rows("Field_2018", condition, "Sep_2018")
flow.datasets.date_range("Sep_2018", "2018-06-01", "2020-01-01")
flow.datasets.date_range("Field_2018", "2018-06-01", "2020-01-01")

In [ ]:
# convert spark dataframes to pandas dataset
flow.datasets.make_dataset("Sep", label="Qliq", pandas=True)
flow.datasets.make_dataset("Field", label="Qliq", pandas=True)
flow.datasets.make_dataset("Sep_2018", label="Qliq", pandas=True)
flow.datasets.make_dataset("Field_2018", label="Qliq", pandas=True)
flow.datasets.make_dataset("OrigSep", label="Qliq", pandas=True)
flow.datasets.make_dataset("OrigField", label="Qliq", pandas=True)
flow.datasets.make_dataset("OrigSep_2018", label="Qliq", pandas=True)
flow.datasets.make_dataset("OrigField_2018", label="Qliq", pandas=True)

flow.next_phase()

In [ ]:
# 
datasets = flow.dataexplore.get_datasets()
eval_preds = flow.dataexplore.fitpredict_naivemodels(datasets["OrigSep_2018"], datasets["OrigField_2018"], eval_all=True)
flow.dataexplore.plotmodels(datasets["OrigSep_2018"], datasets["OrigField_2018"], eval_preds)